# Lab 3. Quantization and Pruning

Sometimes you may want to run your model on edge devices such as Raspberry Pi's, mobile phones or IoT devices. These devices have limited computational resources and memory. Therefore, you need to optimize your model to run on these devices. In this lab, you will learn about some of the techniques that can be used to optimize your model for edge devices.

You will start by training a Keras model. Then you will use this model as a baseline for comparison with the reduced models. You will go through the following techniques of model size reduction (the links contain guides about each technique):

* [post-training quantization](https://www.tensorflow.org/lite/performance/post_training_quantization)
* [quantization aware training](https://www.tensorflow.org/model_optimization/guide/quantization/training_comprehensive_guide)
* [weight pruning](https://www.tensorflow.org/model_optimization/guide/pruning/comprehensive_guide)

## Part 0. Preparation

Let's first install a fixed TensorFlow version, the TensorFlow Model Optimization Toolkit, and the LiteRT package.

In [18]:
!pip install tensorflow==2.19.0 tensorflow_model_optimization==0.8.0 ai-edge-litert

Then import a few common libraries that we'll use throughout the lab. Print the version of TensorFlow to check if everything is working correctly.

We'll use the following libraries:

- `tensorflow`: to do the actual ML
- `ai_edge_litert`: to convert the model to LiteRT format (formerly TensorFlow Lite)
- `numpy`: for handling the dataset (MNIST)
- `os`: for reading model files size
- `tempfile`: to create temporary files (for saving gzipped models)
- `zipfile`: to make a gzipped model

In [2]:
import tensorflow as tf
#from ai_edge_litert.interpreter import Interpreter
import numpy as np
import os
import tempfile
import zipfile

print(tf.__version__)

2.14.1


Let's also define a few string constants and utility functions to make our code easier to maintain.

In [3]:
# String constants for model filenames
FILE_WEIGHTS = 'baseline.weights.h5'
FILE_NON_QUANTIZED = 'non_quantized.keras'
FILE_NON_QUANTIZED_TFLITE = 'non_quantized.tflite'
FILE_PT_QUANTIZED = 'post_training_quantized.tflite'
FILE_QAT_QUANTIZED = 'quant_aware_quantized.tflite'
FILE_PRUNED_MODEL = 'pruned_model.keras'
FILE_PRUNED_QUANTIZED_TFLITE = 'pruned_quantized.tflite'
FILE_PRUNED_NON_QUANTIZED_TFLITE = 'pruned_non_quantized.tflite'

# Dictionaries to hold measurements
MODEL_SIZE = {}
ACCURACY = {}

# Constants for training
BATCH_SIZE = 32
EPOCHS = 2
SHUFFLE = False
VALIDATION_SPLIT = 0.1

In [4]:
def print_metric(metric_dict, metric_name):
  '''Prints key and values stored in a dictionary'''
  for metric, value in metric_dict.items():
    print(f'{metric_name} for {metric}: {value}')


def model_builder():
  '''Returns a shallow CNN for training on the MNIST dataset'''

  keras = tf.keras

  # Define the model architecture.
  model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(28, 28)), # Input layer
    keras.layers.Reshape(target_shape=(28, 28, 1)), # Reshape to 28x28x1
    keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'), # Input layer
    keras.layers.MaxPooling2D(pool_size=(2, 2)), # Downsizing images
    keras.layers.Flatten(), # Images are 3D, flatten to 1D
    keras.layers.Dense(10, activation='softmax') # Output layer (with 10 possible outputs for each digit)
  ])

  return model


def evaluate_tflite_model(filename, x_test, y_test):
  '''
  Measures the accuracy of a given TF Lite model and test set

  Args:
    filename (string) - filename of the model to load
    x_test (numpy array) - test images
    y_test (numpy array) - test labels

  Returns
    float showing the accuracy against the test set
  '''

  # Initialize the TF Lite Interpreter and allocate tensors
  interpreter = tf.lite.Interpreter(model_path=filename)
  interpreter.allocate_tensors()

  # Get input and output index
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Initialize empty predictions list
  prediction_digits = []

  # Run predictions on every image in the "test" dataset.
  for i, test_image in enumerate(x_test):
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == y_test).mean()

  return accuracy


def get_gzipped_model_size(file):
  '''Returns size of gzipped model, in bytes.'''
  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

### Download and prepare the MNIST dataset

You will be using the [MNIST](https://keras.io/api/datasets/mnist/) dataset which is hosted in [Keras Datasets](https://keras.io/api/datasets/). The MNIST dataset contains 60,000 training images of handwritten digits from zero to nine and 10,000 images for testing. The images are grayscale and 28 x 28 pixels. You will use this dataset to train a model to classify the images into 10 classes (one for each digit).

In [5]:
# Load MNIST dataset
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

11490434/11490434 [==============================] - 1s 0us/step


## Part 1. Baseline model

Let's first build and train a Keras model. This will be the baseline where you will be comparing the mobile optimized versions later on. This will just be a shallow CNN with a softmax output to classify a given MNIST digit. You can review the `model_builder()` function in the utilities at the top of this notebook but we also printed the model summary below to show the architecture.

You will also save the weights so you can reinitialize the other models later the same way. This is not needed in real projects but for this demo notebook, it would be good to have the same initial state later so you can compare the effects of the optimizations.

In [6]:
# Create the baseline model
baseline_model = model_builder()

# Save the initial weights for use later
baseline_model.save_weights(FILE_WEIGHTS)

# Print the model summary
baseline_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 28, 28, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 12)        120       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 12)        0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 2028)              0         
                                                                 
 dense (Dense)               (None, 10)                20290     
                                                                 
Total params: 20410 (79.73 KB)
Trainable params: 20410 (79.73 KB)
Non-trainable params: 0 (0.00 Byte)
____________________

Now, compile and train the model. In practice, it's best to shuffle the train set but for this lab, it is set to `False` for reproducibility of the results. One epoch will reach around 91% accuracy.

In [7]:
# Setup the model for training
baseline_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
baseline_model.fit(train_images, train_labels, epochs=EPOCHS,
                   validation_split=VALIDATION_SPLIT, shuffle=SHUFFLE)

Epoch 1/2
1688/1688 [==============================] - 6s 3ms/step - loss: 0.3083 - accuracy: 0.9139 - val_loss: 0.1214 - val_accuracy: 0.9695
Epoch 2/2
1688/1688 [==============================] - 5s 3ms/step - loss: 0.1212 - accuracy: 0.9658 - val_loss: 0.0817 - val_accuracy: 0.9798


Let's save the accuracy of the model against the test set so you can compare later.

In [8]:
# Get the baseline accuracy
_, ACCURACY['baseline Keras model'] = baseline_model.evaluate(test_images, test_labels)

313/313 [==============================] - 0s 1ms/step - loss: 0.0894 - accuracy: 0.9731


Next, you will save the Keras model as a file and record its size as well.

In [9]:
# Save the Keras model
baseline_model.save(FILE_NON_QUANTIZED)

# Save and get the model size
MODEL_SIZE['baseline'] = os.path.getsize(FILE_NON_QUANTIZED)

# Print records so far
print_metric(ACCURACY, "test accuracy")
print_metric(MODEL_SIZE, "model size in bytes")

test accuracy for baseline Keras model: 0.9731000065803528
model size in bytes for baseline: 271215


### Convert the model to TF Lite format

Next, you will convert the model to [Tensorflow Lite (TF Lite)](https://www.tensorflow.org/lite/guide) format. This is designed to make Tensorflow models more efficient and lightweight when running on mobile, embedded, and IoT devices.

You can convert a Keras model with TF Lite's [Converter](https://www.tensorflow.org/lite/convert/index) class and we've incorporated it in the short helper function below. Notice that there is a `quantize` flag which you can use to quantize the model.

Complete the function below:

1. Create a TFLiteConverter for the given model.
2. Add default optimization when the parameter `quantize` is `True`.
3. Convert the model.

Check the [documentation](https://www.tensorflow.org/lite/performance/post_training_quantization) for more information.

In [10]:
def convert_tflite(model, filename, quantize=False):
  '''
  Converts the model to TF Lite format and writes to a file

  Args:
    model (Keras model) - model to convert to TF Lite
    filename (string) - string to use when saving the file
    quantize (bool) - flag to indicate quantization

  Returns:
    None
  '''

  # Initialize the converter
  # TODO: create a TFLite converter
  converter = tf.lite.TFLiteConverter.from_keras_model(model)

  # Set for quantization if flag is set to True
  if quantize:
    # TODO: Add DEFAULT TFLite optimizer
    converter.optimizations = [tf.lite.Optimize.DEFAULT]

  # Convert the model
  # TODO: start the model conversion
  tflite_model = converter.convert()

  # Save the model.
  with open(filename, 'wb') as f:
    f.write(tflite_model)

You will use the helper function to convert the Keras model then get its size and accuracy. Take note that this is *not yet* quantized.

In [11]:
# Convert baseline model
convert_tflite(baseline_model, FILE_NON_QUANTIZED_TFLITE)

INFO:tensorflow:Assets written to: C:\Users\peelm\AppData\Local\Temp\tmpv0ul93ja\assets


INFO:tensorflow:Assets written to: C:\Users\peelm\AppData\Local\Temp\tmpv0ul93ja\assets


You will notice that there is already a slight decrease in model size when converting to `.tflite` format.

In [12]:
MODEL_SIZE['non quantized tflite'] = os.path.getsize(FILE_NON_QUANTIZED_TFLITE)

print_metric(MODEL_SIZE, 'model size in bytes')

model size in bytes for baseline: 271215
model size in bytes for non quantized tflite: 85012


The accuracy will also be nearly identical when converting between formats. You can setup a TF Lite model for input-output using its [Interpreter](https://www.tensorflow.org/api_docs/python/tf/lite/Interpreter) class. This is shown in the `evaluate_tflite_model()` helper function provided in the `Utilities` section earlier.

*Note: If you see a `Runtime Error: There is at least 1 reference to internal data in the interpreter in the form of a numpy array or slice.` , please try re-running the cell.*

In [13]:
ACCURACY['non quantized tflite'] = evaluate_tflite_model(FILE_NON_QUANTIZED_TFLITE, test_images, test_labels)
print_metric(ACCURACY, 'test accuracy')

test accuracy for baseline Keras model: 0.9731000065803528
test accuracy for non quantized tflite: 0.9731


## Part 2. Post-Training Quantization

Now that you have the baseline metrics, you can now observe the effects of quantization. This process involves converting floating point representations into integer to reduce model size and achieve faster computation.

As shown in the `convert_tflite()` helper function earlier, you can easily do [post-training quantization](https://www.tensorflow.org/lite/performance/post_training_quantization) with the TF Lite API. You just need to set the converter optimization and assign an [Optimize](https://www.tensorflow.org/api_docs/python/tf/lite/Optimize) Enum.

You will set the `quantize` flag to do that and get the metrics again.

In [14]:
# Convert and quantize the baseline model
convert_tflite(baseline_model, FILE_PT_QUANTIZED, quantize=True)

INFO:tensorflow:Assets written to: C:\Users\peelm\AppData\Local\Temp\tmpii4ftaya\assets


INFO:tensorflow:Assets written to: C:\Users\peelm\AppData\Local\Temp\tmpii4ftaya\assets


In [15]:
# Get the model size
MODEL_SIZE['post training quantized tflite'] = os.path.getsize(FILE_PT_QUANTIZED)

print_metric(MODEL_SIZE, 'model size')

model size for baseline: 271215
model size for non quantized tflite: 85012
model size for post training quantized tflite: 24256


You should see around a 4X reduction in model size in the quantized version. This comes from converting the 32 bit representations (float) into 8 bits (integer).



In [16]:
ACCURACY['post training quantized tflite'] = evaluate_tflite_model(FILE_PT_QUANTIZED, test_images, test_labels)

print_metric(ACCURACY, 'test accuracy')

test accuracy for baseline Keras model: 0.9731000065803528
test accuracy for non quantized tflite: 0.9731
test accuracy for post training quantized tflite: 0.9735


You can expect the accuracy to not be the same when quantizing the model. Most of the time it will decrease but in some cases, it can even increase. Again, this can be attributed to the loss of precision when you remove the extra bits from the float data.

## Part 3. Quantization Aware Training

When post-training quantization results in loss of accuracy that is unacceptable for your application, you can consider doing [quantization aware training](https://www.tensorflow.org/model_optimization/guide/quantization/training) before quantizing the model. This simulates the loss of precision by inserting fake quant nodes in the model during training. That way, your model will learn to adapt with the loss of precision to get more accurate predictions.

The [Tensorflow Model Optimization Toolkit](https://www.tensorflow.org/model_optimization) provides a [quantize_model()](https://www.tensorflow.org/model_optimization/api_docs/python/tfmot/quantization/keras/quantize_model) method to do this quickly. You will build the baseline model again but this time, pass it into the `quantize_model()` method to indicate quantization aware training (note the `TODO`). Check the [documentation](https://www.tensorflow.org/model_optimization/guide/quantization/training_comprehensive_guide) for more information.

Take note that in case you decide to pass in a model that is already trained, then make sure to recompile before you continue training.

In [17]:
import tensorflow_model_optimization as tfmot

# Define the model architecture.
model_to_quantize = model_builder()

# Reinitialize weights with saved file
model_to_quantize.load_weights(FILE_WEIGHTS)

# Quantize the model
# TODO: create a quantization aware model from model_to_quantize
q_aware_model = tfmot.quantization.keras.quantize_model(model_to_quantize)

# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

q_aware_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer (QuantizeLa  (None, 28, 28)            3         
 yer)                                                            
                                                                 
 quant_reshape_1 (QuantizeW  (None, 28, 28, 1)         1         
 rapperV2)                                                       
                                                                 
 quant_conv2d_1 (QuantizeWr  (None, 26, 26, 12)        147       
 apperV2)                                                        
                                                                 
 quant_max_pooling2d_1 (Qua  (None, 13, 13, 12)        1         
 ntizeWrapperV2)                                                 
                                                                 
 quant_flatten_1 (QuantizeW  (None, 2028)             

You may have noticed a slight difference in the model summary above compared to the baseline model summary in the earlier sections. The total params count increased as expected because of the nodes added by the `quantize_model()` method.

With that, you can now train the model. You will notice that the accuracy is a bit lower because the model is simulating the loss of precision. The training will take a bit longer if you want to achieve the same training accuracy as the earlier run. For this exercise though, we will keep to 1 epoch.

In [18]:
# Train the model
q_aware_model.fit(train_images, train_labels, epochs=EPOCHS,
                  validation_split=VALIDATION_SPLIT, shuffle=SHUFFLE)

Epoch 1/2
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3129 - accuracy: 0.9126 - val_loss: 0.1264 - val_accuracy: 0.9680
Epoch 2/2
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1265 - accuracy: 0.9644 - val_loss: 0.0852 - val_accuracy: 0.9785


You can then get the accuracy of the Keras model before and after quantizing the model. The accuracy is expected to be nearly identical because the model is trained to counter the effects of quantization.

In [19]:
# Get the accuracy of the quantization aware trained model (not yet quantized)
_, ACCURACY['quantization aware non-quantized'] = q_aware_model.evaluate(test_images, test_labels, verbose=0)
print_metric(ACCURACY, 'test accuracy')

test accuracy for baseline Keras model: 0.9731000065803528
test accuracy for non quantized tflite: 0.9731
test accuracy for post training quantized tflite: 0.9735
test accuracy for quantization aware non-quantized: 0.9732999801635742


In [20]:
# Convert and quantize the model.
convert_tflite(q_aware_model, FILE_QAT_QUANTIZED, quantize=True)

# Get the accuracy of the quantized model
ACCURACY['quantization aware quantized'] = evaluate_tflite_model(FILE_QAT_QUANTIZED, test_images, test_labels)
print_metric(ACCURACY, 'test accuracy')

INFO:tensorflow:Assets written to: C:\Users\peelm\AppData\Local\Temp\tmp7ejypp04\assets


INFO:tensorflow:Assets written to: C:\Users\peelm\AppData\Local\Temp\tmp7ejypp04\assets
c:\MLOps\resources\03-quantization-and-pruning\tfmot-env\lib\site-packages\tensorflow\lite\python\convert.py:947: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


test accuracy for baseline Keras model: 0.9731000065803528
test accuracy for non quantized tflite: 0.9731
test accuracy for post training quantized tflite: 0.9735
test accuracy for quantization aware non-quantized: 0.9732999801635742
test accuracy for quantization aware quantized: 0.9733


Let's also check the model size after quantizing the model. You should see about the same size as the post-training quantized model.

In [21]:
# Get the model size of the quantized model
MODEL_SIZE['quantization aware quantized tflite'] = os.path.getsize(FILE_QAT_QUANTIZED)

print_metric(MODEL_SIZE, 'model size')

model size for baseline: 271215
model size for non quantized tflite: 85012
model size for post training quantized tflite: 24256
model size for quantization aware quantized tflite: 25264


## Part 4. Pruning

Let's now move on to another technique for reducing model size: [Pruning](https://www.tensorflow.org/model_optimization/guide/pruning/pruning_with_keras). This process involves zeroing out insignificant (i.e. low magnitude) weights. The intuition is these weights do not contribute as much to making predictions so you can remove them and get the same result. Making the weights sparse helps in compressing the model more efficiently and you will see that in this section.

The Tensorflow Model Optimization Toolkit again has a convenience method for this. The [prune_low_magnitude()](https://www.tensorflow.org/model_optimization/api_docs/python/tfmot/sparsity/keras/prune_low_magnitude) method puts wrappers in a Keras model so it can be pruned during training. You will pass in the baseline model that you already trained earlier. You will notice that the model summary show increased params because of the wrapper layers added by the pruning method.

You can set how the pruning is done during training. Below, you will use [PolynomialDecay](https://www.tensorflow.org/model_optimization/api_docs/python/tfmot/sparsity/keras/PolynomialDecay) to indicate how the sparsity ramps up with each step. Another option available in the library is [Constant Sparsity](https://www.tensorflow.org/model_optimization/api_docs/python/tfmot/sparsity/keras/ConstantSparsity). Use the following parameters for `PolynomialDecay`:

- initial sparsity: 0.50
- final sparsity: 0.80
- begin step: 0
- end step: use `end_step` variable calculated in this cell

Check the [documentation](https://www.tensorflow.org/model_optimization/guide/pruning/comprehensive_guide) for more information.

In [22]:
# Get the pruning method
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

num_images = train_images.shape[0] * (1 - VALIDATION_SPLIT)
end_step = np.ceil(num_images / BATCH_SIZE).astype(np.int32) * EPOCHS

# Define pruning schedule.
# TODO: replace None with PolynomialDecay with the above parameters
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.50,  # start at 50% sparsity
        final_sparsity=0.80,    # target 80% sparsity
        begin_step=0,           # start pruning immediately
        end_step=end_step,      # stop pruning at the last training step
        frequency=100           # update pruning every 100 steps
    )
}

# Define the model architecture.
model_to_prune = model_builder()

# Reinitialize weights with saved file
model_to_prune.load_weights(FILE_WEIGHTS)

# TODO: wrap model_to_prune with the pruning method (prune_low_magnitude), you might need ** to pass pruning_params
model_for_pruning = prune_low_magnitude(model_to_prune, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model_for_pruning.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_reshap  (None, 28, 28, 1)         1         
 e_2 (PruneLowMagnitude)                                         
                                                                 
 prune_low_magnitude_conv2d  (None, 26, 26, 12)        230       
 _2 (PruneLowMagnitude)                                          
                                                                 
 prune_low_magnitude_max_po  (None, 13, 13, 12)        1         
 oling2d_2 (PruneLowMagnitu                                      
 de)                                                             
                                                                 
 prune_low_magnitude_flatte  (None, 2028)              1         
 n_2 (PruneLowMagnitude)                                         
                                                      

You can also peek at the weights of one of the layers in your model. After pruning, you will notice that many of these will be zeroed out.

In [23]:
# Preview model weights
model_for_pruning.weights[1]

<tf.Variable 'conv2d_2/kernel:0' shape=(3, 3, 1, 12) dtype=float32, numpy=
array([[[[-0.1874944 ,  0.18834634, -0.15585211, -0.18766809,
           0.09313317,  0.15306957, -0.06051186,  0.08744045,
          -0.04322577,  0.17929526, -0.1215722 , -0.07562938]],

        [[-0.1999019 ,  0.07982822,  0.02184688,  0.07378893,
           0.06320061,  0.05627571, -0.09883521,  0.0589857 ,
           0.20760892,  0.00038539,  0.08982269,  0.10169779]],

        [[-0.21125436,  0.09592672, -0.04536927, -0.15225495,
          -0.07508099,  0.1524842 , -0.20695427,  0.09132765,
          -0.15947379,  0.00738372,  0.1277401 , -0.07517678]]],


       [[[-0.02989124,  0.08846109, -0.07055584,  0.1970266 ,
           0.11983071,  0.01264144, -0.19520293,  0.10918759,
          -0.06121914,  0.0896766 ,  0.05206241,  0.15553336]],

        [[ 0.18553008,  0.03859188, -0.21454436,  0.12254961,
          -0.05493769,  0.21142204, -0.0148548 , -0.07752323,
           0.04671215,  0.12096669,  0.2157

With that, you can now start re-training the model. Add the [UpdatePruningStep()](https://www.tensorflow.org/model_optimization/api_docs/python/tfmot/sparsity/keras/UpdatePruningStep) callback to the `callbacks` array.

In [24]:
# Callback to update pruning wrappers at each step
# TODO: add UpdatePruningStep callback
callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep()
]

# Train and prune the model
model_for_pruning.fit(train_images, train_labels, epochs=EPOCHS,
                      validation_split=VALIDATION_SPLIT, shuffle=SHUFFLE,
                      callbacks=callbacks)

Epoch 1/2
1688/1688 [==============================] - 7s 3ms/step - loss: 0.3918 - accuracy: 0.8892 - val_loss: 0.2236 - val_accuracy: 0.9392
Epoch 2/2
1688/1688 [==============================] - 5s 3ms/step - loss: 0.2553 - accuracy: 0.9266 - val_loss: 0.1853 - val_accuracy: 0.9492


Now see how the weights in the same layer looks like after pruning.

In [25]:
# Preview model weights
model_for_pruning.weights[1]

<tf.Variable 'conv2d_2/kernel:0' shape=(3, 3, 1, 12) dtype=float32, numpy=
array([[[[-0.        ,  0.59820426,  0.        ,  0.        ,
           0.7096379 ,  0.        , -0.        ,  0.        ,
           0.        ,  0.6498669 ,  0.        ,  0.        ]],

        [[-0.        ,  0.        ,  0.        ,  0.        ,
           0.        ,  0.        , -0.        ,  0.        ,
           0.8160668 ,  0.        ,  0.        ,  0.        ]],

        [[-0.        ,  0.        ,  0.        , -0.        ,
          -0.        ,  0.        , -1.0130419 ,  0.        ,
           0.        ,  0.        ,  0.        ,  0.        ]]],


       [[[-0.        ,  0.        ,  0.        ,  0.70149523,
           0.8649215 ,  0.        , -0.        ,  0.        ,
           0.        ,  0.        ,  0.        ,  0.673383  ]],

        [[ 0.5607074 ,  0.        ,  0.        ,  0.658833  ,
           0.        ,  0.7183981 , -0.        , -0.        ,
           0.        ,  0.6513123 ,  0.7174

After pruning, you can remove the wrapper layers to have the same layers and params as the baseline model. You can do that with the [strip_pruning()](https://www.tensorflow.org/model_optimization/api_docs/python/tfmot/sparsity/keras/strip_pruning) method as shown below. You will do this so you can save the model and also export to TF Lite format just like in the previous sections.

In [26]:
# Remove pruning wrappers
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)
model_for_export.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_2 (Reshape)         (None, 28, 28, 1)         0         
                                                                 
 conv2d_2 (Conv2D)           (None, 26, 26, 12)        120       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 13, 13, 12)        0         
 g2D)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 2028)              0         
                                                                 
 dense_2 (Dense)             (None, 10)                20290     
                                                                 
Total params: 20410 (79.73 KB)
Trainable params: 20410 (79.73 KB)
Non-trainable params: 0 (0.00 Byte)
__________________

You will see the same model weights but the index is different because the wrappers were removed.

In [27]:
# Preview model weights (index 1 earlier is now 0 because pruning wrappers were removed)
model_for_export.weights[0]

<tf.Variable 'conv2d_2/kernel:0' shape=(3, 3, 1, 12) dtype=float32, numpy=
array([[[[-0.        ,  0.59820426,  0.        ,  0.        ,
           0.7096379 ,  0.        , -0.        ,  0.        ,
           0.        ,  0.6498669 ,  0.        ,  0.        ]],

        [[-0.        ,  0.        ,  0.        ,  0.        ,
           0.        ,  0.        , -0.        ,  0.        ,
           0.8160668 ,  0.        ,  0.        ,  0.        ]],

        [[-0.        ,  0.        ,  0.        , -0.        ,
          -0.        ,  0.        , -1.0130419 ,  0.        ,
           0.        ,  0.        ,  0.        ,  0.        ]]],


       [[[-0.        ,  0.        ,  0.        ,  0.70149523,
           0.8649215 ,  0.        , -0.        ,  0.        ,
           0.        ,  0.        ,  0.        ,  0.673383  ]],

        [[ 0.5607074 ,  0.        ,  0.        ,  0.658833  ,
           0.        ,  0.7183981 , -0.        , -0.        ,
           0.        ,  0.6513123 ,  0.7174

You will notice below that the pruned model will have the same file size as the baseline_model when saved. This is to be expected. The improvement will be noticeable when you compress the model as will be shown in the cell after this.

In [28]:
# Save Keras model
model_for_export.save(FILE_PRUNED_MODEL)

# Get uncompressed model size of baseline and pruned models
MODEL_SIZE['pruned non quantized'] = os.path.getsize(FILE_PRUNED_MODEL)

print_metric(MODEL_SIZE, 'model_size in bytes')

model_size in bytes for baseline: 271215
model_size in bytes for non quantized tflite: 85012
model_size in bytes for post training quantized tflite: 24256
model_size in bytes for quantization aware quantized tflite: 25264
model_size in bytes for pruned non quantized: 97135


You will use the `get_gzipped_model_size()` helper function in the `Utilities` to compress the models and get its resulting file size. You will notice that the pruned model is about 3 times smaller. This is because of the sparse weights generated by the pruning process. The zeros can be compressed much more efficiently than the low magnitude weights before pruning.

In [29]:
# Get compressed size of baseline and pruned models
MODEL_SIZE['baseline (gzipped)'] = get_gzipped_model_size(FILE_NON_QUANTIZED)
MODEL_SIZE['pruned non quantized (gzipped)'] = get_gzipped_model_size(FILE_PRUNED_MODEL)

print_metric(MODEL_SIZE, "gzipped model size in bytes")

gzipped model size in bytes for baseline: 271215
gzipped model size in bytes for non quantized tflite: 85012
gzipped model size in bytes for post training quantized tflite: 24256
gzipped model size in bytes for quantization aware quantized tflite: 25264
gzipped model size in bytes for pruned non quantized: 97135
gzipped model size in bytes for baseline (gzipped): 233228
gzipped model size in bytes for pruned non quantized (gzipped): 25637


You can make the model even more lightweight by quantizing the pruned model. This achieves around 10X reduction in compressed model size as compared to the baseline.

In [30]:
# Convert and quantize the pruned model.
pruned_quantized_tflite = convert_tflite(model_for_export, FILE_PRUNED_QUANTIZED_TFLITE, quantize=True)

# Compress and get the model size
MODEL_SIZE['pruned quantized tflite (gzipped)'] = get_gzipped_model_size(FILE_PRUNED_QUANTIZED_TFLITE)
print_metric(MODEL_SIZE, "gzipped model size in bytes")

INFO:tensorflow:Assets written to: C:\Users\peelm\AppData\Local\Temp\tmpnn2e9z5v\assets


INFO:tensorflow:Assets written to: C:\Users\peelm\AppData\Local\Temp\tmpnn2e9z5v\assets


gzipped model size in bytes for baseline: 271215
gzipped model size in bytes for non quantized tflite: 85012
gzipped model size in bytes for post training quantized tflite: 24256
gzipped model size in bytes for quantization aware quantized tflite: 25264
gzipped model size in bytes for pruned non quantized: 97135
gzipped model size in bytes for baseline (gzipped): 233228
gzipped model size in bytes for pruned non quantized (gzipped): 25637
gzipped model size in bytes for pruned quantized tflite (gzipped): 8455


As expected, the TF Lite model's accuracy will also be close to the Keras model.

In [31]:
# Get accuracy of pruned Keras and TF Lite models
_, ACCURACY['pruned model'] = model_for_pruning.evaluate(test_images, test_labels)
ACCURACY['pruned and quantized tflite'] = evaluate_tflite_model(FILE_PRUNED_QUANTIZED_TFLITE, test_images, test_labels)

print_metric(ACCURACY, 'accuracy')

313/313 [==============================] - 0s 1ms/step - loss: 0.2238 - accuracy: 0.9366
accuracy for baseline Keras model: 0.9731000065803528
accuracy for non quantized tflite: 0.9731
accuracy for post training quantized tflite: 0.9735
accuracy for quantization aware non-quantized: 0.9732999801635742
accuracy for quantization aware quantized: 0.9733
accuracy for pruned model: 0.9366000294685364
accuracy for pruned and quantized tflite: 0.936
